In [1]:
import os
import pandas as pd

data_location = r'D:\Datasets\gender-names\gender_refine-csv.csv'

data = pd.read_csv(data_location)
for i, gender in enumerate(data["Gender"]):
  if gender == 3:
    data.drop(data.index[0], inplace = True)
data['length'] = data.apply(lambda row: len(row.Name), axis=1)

vowels = 'aeiou'
data['no_of_vowels'] = 0
data['percentage_of_vowels'] = 0.0
data['ends_with_vowels'] = 0

for i, names in enumerate(data['Name']):
  count_vowels = 0
  for alpha in names.lower():
    if alpha in vowels:
      count_vowels += 1
  data['no_of_vowels'][i] = count_vowels
  data['percentage_of_vowels'][i] = count_vowels/(len(names))
  if names[len(names)-1] in vowels:
    data['ends_with_vowels'][i] = 1
  else:
    data['ends_with_vowels'][i] = 0
    
maxLength = data['length'].max()

initial = 'abcdefghijklm'
final = 'nopqrstuvwxyz'
dict_alpha = {}
for i in (initial+final):
  if i in initial:
    dict_alpha[i] = float(ord(i) -96 -14)
  if i in final:
    dict_alpha[i] = float(ord(i) -96 -13)
dict_alpha["'"] = 0.0
dict_alpha["-"] = 0.0
dict_alpha[","] = 0.0
dict_alpha[" "] = 0.0
dict_alpha["."] = 0.0
dict_alpha["\\"] = 0.0
dict_alpha["/"] = 0.0
dict_alpha["("] = 0.0
dict_alpha[")"] = 0.0
dict_alpha["["] = 0.0
dict_alpha["]"] = 0.0
dict_alpha["{"] = 0.0
dict_alpha["}"] = 0.0

for i in range(10):
  dict_alpha[str(i)] = 0.0


for i in range(22):
  data["{}st_character".format(i)] = 0

for i, names in enumerate(data['Name']):
  for j in range(maxLength):
    if j < len(names):
      data["{}st_character".format(j)][i] = dict_alpha[names[j].lower()]
    else:
      data["{}st_character".format(j)][i] = 0.0
    
data = data.sample(frac=1).reset_index(drop=True)
y = data['Gender']
data.drop(labels="Gender", axis=1, inplace=True)

data.drop('Name', inplace=True, axis=1)

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier

X_train = data.values
y_train = y

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)


D:\Anaconda\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

In [2]:
state = 12
val_size = 0.3

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,
                                                 test_size=val_size,
                                                 shuffle = True)

In [4]:
data.head(15)
data_scoreless = data.drop('Score', axis=1)

In [5]:
X = data.values

state = 2
val_size = 0.3

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,
                                                 test_size=val_size,
                                                 shuffle=True)

In [9]:
gb_clf_final = GradientBoostingClassifier(learning_rate=0.1,
                                         n_estimators=70,
                                         max_features='sqrt',
                                         max_depth=5,
                                         min_samples_split=1000,
                                         min_samples_leaf=50,
                                         verbose=1)

gb_clf_final.fit(X_train, y_train)
print("Accuracy score (training):{0:.3f}".format(gb_clf_final.score(X_train, y_train)))
print("Accuracy score (validation):{0:.3f}".format(gb_clf_final.score(X_val, y_val)))

      Iter       Train Loss   Remaining Time 
         1       40472.4877           11.08s
         2       40162.7690           11.73s
         3       40004.9355           10.49s
         4       39873.5971           10.45s
         5       39736.6613            9.98s
         6       39640.9430            9.62s
         7       39528.6524            9.41s
         8       39434.0391            9.58s
         9       39357.0313            9.30s
        10       39300.0341            9.14s
        20       39021.6257            7.41s
        30       38915.5729            5.84s
        40       38861.4757            4.41s
        50       38829.3147            2.90s
        60       38808.1862            1.45s
        70       38793.0504            0.00s
Accuracy score (training):0.622
Accuracy score (validation):0.616


In [11]:
learning_rate_list = [0.05, 0.075, 0.1, 0.5, 0.75, 1]

for learning_rate in learning_rate_list:
  gb_clf_final = GradientBoostingClassifier(learning_rate=learning_rate,
                                           n_estimators=70,
                                           max_features='sqrt',
                                           max_depth=5,
                                           min_samples_split=1000,
                                           min_samples_leaf=50,
                                           verbose=1)
  gb_clf_final.fit(X_train, y_train)
  
  print("Learning Rate :", learning_rate)
  print("Accuracy score (training):{0:.3f}".format(gb_clf_final.score(X_train, y_train)))
  print("Accuracy score (validation):{0:.3f}".format(gb_clf_final.score(X_val, y_val)))

      Iter       Train Loss   Remaining Time 
         1       40666.0577            9.84s
         2       40579.9448            9.49s
         3       40440.1321            9.44s
         4       40314.0627           10.10s
         5       40208.6625           10.04s
         6       40102.1882            9.69s
         7       40007.6860            9.43s
         8       39918.2103            9.41s
         9       39851.2671            9.11s
        10       39771.9930            9.08s
        20       39347.1664            7.58s
        30       39140.4926            7.34s
        40       39028.4154            5.40s
        50       38965.7614            3.57s
        60       38919.3987            1.76s
        70       38888.5518            0.00s
Learning Rate : 0.05
Accuracy score (training):0.621
Accuracy score (validation):0.616
      Iter       Train Loss   Remaining Time 
         1       40620.4812           10.46s
         2       40382.8362           10.75s
         3 

In [13]:
learning_rate = 0.5
n_estimators_list = [20, 40, 60, 70, 80, 100]
for n_estimators in n_estimators_list:
  gb_clf_final = GradientBoostingClassifier(learning_rate = learning_rate,
                                           n_estimators = n_estimators,
                                           max_features = 'sqrt',
                                           max_depth = 5,
                                           min_samples_split=1000,
                                           min_samples_leaf=50,
                                           verbose=1,
                                           warm_start = True)
  gb_clf_final.fit(X_train, y_train)
  print("N estimators", n_estimators)
  print("Accuracy score (training):{0:.3f}".format(gb_clf_final.score(X_train, y_train)))
  print("Accuracy score (validation):{0:.3f}".format(gb_clf_final.score(X_val, y_val)))

      Iter       Train Loss   Remaining Time 
         1       39622.5375            2.65s
         2       39393.1197            2.51s
         3       39152.8232            2.28s
         4       39058.5246            2.30s
         5       39002.7267            2.22s
         6       38939.6756            2.13s
         7       38910.6834            1.97s
         8       38889.8403            1.80s
         9       38862.4872            1.63s
        10       38847.9730            1.49s
        20       38762.3249            0.00s
N estimators 20
Accuracy score (training):0.622
Accuracy score (validation):0.616
      Iter       Train Loss   Remaining Time 
         1       40175.4197            5.41s
         2       55309.3686            5.29s
         3       55132.5172            5.03s
         4       55028.0495            4.86s
         5       54977.5313            4.77s
         6       54940.6385            4.59s
         7       54915.0791            4.39s
         8      

In [18]:
learning_rate = 0.5
n_estimators = 60
max_depth_list = [17, 19, 22, 25]
for max_depth in max_depth_list:
  gb_clf_final = GradientBoostingClassifier(learning_rate = learning_rate,
                                           n_estimators = n_estimators,
                                           max_depth = max_depth,
                                           max_features = 'sqrt',
                                           min_samples_split = 1000,
                                           min_samples_leaf = 50,
                                           verbose = 1,
                                           warm_start = True)
  gb_clf_final.fit(X_train, y_train)
  
  print("Max Depth: ", max_depth)
  print("Accuracy score (training):{0:.3f}".format(gb_clf_final.score(X_train, y_train)))
  print("Accuracy score (validation):{0:.3f}".format(gb_clf_final.score(X_val, y_val)))

      Iter       Train Loss   Remaining Time 
         1       39893.3590            9.47s
         2       47762.1786           10.82s
         3       47569.7890           10.55s
         4       47432.5729           11.21s
         5       47361.1943           11.09s
         6       47306.8250           10.87s
         7       47269.5054           10.89s
         8       47240.7456           10.66s
         9       47217.4894           10.58s
        10       47005.3733           10.35s
        20       47238.2375            8.30s
        30 1146693017638430019258050732176522664008154886673193620405940756230102284929413819175762701047471240380416.0000            6.30s
        40 1146693017638430019258050732176522664008154886673193620405940756230102284929413819175762701047471240380416.0000            4.25s
        50 1146693017638430019258050732176522664008154886673193620405940756230102284929413819175762701047471240380416.0000            2.25s
        60 1146693017638430019258050732

In [19]:
learning_rate = 0.5
n_estimators = 60
max_depth = 22
min_samples_split_list = [250, 500, 1000, 1500, 2000, 5000]
for min_samples_split in min_samples_split_list:
  gb_clf_final = GradientBoostingClassifier(learning_rate = learning_rate,
                                           n_estimators = n_estimators,
                                           max_depth = max_depth,
                                           max_features = 'sqrt',
                                           min_samples_split = min_samples_split,
                                           min_samples_leaf = 50,
                                           verbose = 1,
                                           warm_start = True)
  gb_clf_final.fit(X_train, y_train)
  
  print("Min Samples Split: ", min_samples_split)
  print("Accuracy score (training):{0:.3f}".format(gb_clf_final.score(X_train, y_train)))
  print("Accuracy score (validation):{0:.3f}".format(gb_clf_final.score(X_val, y_val)))

      Iter       Train Loss   Remaining Time 
         1       39919.7809           10.83s
         2       39280.3482           10.04s
         3       39160.2377           10.71s
         4       39248.7500           10.61s
         5       39173.4199           11.41s
         6       42736.7447           11.64s
         7       42545.2740           11.70s
         8 63871083981811715407872.0000           11.73s
         9 63871083981811715407872.0000           11.47s
        10 63871083981811715407872.0000           11.14s
        20 9385240029760176060099324790363901400082494831050718681679839554009630777132773477676074349674187389012428193792.0000            9.19s
        30 9385240029760176060099324790363901400082494831050718681679839554009630777132773477676074349674187389012428193792.0000            7.03s
        40 9385240029760176060099324790363901400082494831050718681679839554009630777132773477676074349674187389012428193792.0000            4.67s
        50 9385240029760176060

In [21]:
learning_rate = 0.5
n_estimators = 60
max_depth = 22
min_samples_split = 1500
min_samples_leaf_list = 50

gb_clf_final = GradientBoostingClassifier(learning_rate = learning_rate,
                                           n_estimators = n_estimators,
                                           max_depth = max_depth,
                                           max_features = 'sqrt',
                                           min_samples_split = min_samples_split,
                                           min_samples_leaf = min_samples_leaf,
                                           verbose = 1,
                                           warm_start = True)
gb_clf_final.fit(X_train, y_train)
  

print("Accuracy score (training):{0:.3f}".format(gb_clf_final.score(X_train, y_train)))
print("Accuracy score (validation):{0:.3f}".format(gb_clf_final.score(X_val, y_val)))

      Iter       Train Loss   Remaining Time 
         1       39613.1936            9.65s
         2       39294.6905            9.60s
         3       39143.1067            9.25s
         4       39054.8297            9.89s
         5       38981.5418           11.84s
         6       38922.9243           11.88s
         7       38897.0832           11.12s
         8       38870.8167           10.83s
         9       38848.1870           10.47s
        10       38833.0035           10.19s
        20       38689.4811            8.14s
        30       38619.6412            6.06s
        40       38564.5360            4.05s
        50       38524.7569            2.00s
        60       38489.7491            0.00s
Accuracy score (training):0.627
Accuracy score (validation):0.615


In [24]:
from sklearn.feature_extraction import DictVectorizer
vec = DictVectorizer()

data_scoreless.head(10)
data.head(10)

,Score,length,no_of_vowels,percentage_of_vowels,ends_with_vowels,0st_character,1st_character,2st_character,3st_character,4st_character,...,12st_character,13st_character,14st_character,15st_character,16st_character,17st_character,18st_character,19st_character,20st_character,21st_character
0,1.0,7,0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.0,5,0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1.0,5,0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1.0,6,0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1.0,6,0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1.0,5,0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1.0,10,0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,1.0,5,0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,1.0,6,0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1.0,4,0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
X_train

array([[1.  , 0.15, 0.  , ..., 0.  , 0.  , 0.  ],
       [1.  , 0.3 , 0.  , ..., 0.  , 0.  , 0.  ],
       [1.  , 0.3 , 0.  , ..., 0.  , 0.  , 0.  ],
       ...,
       [0.  , 0.7 , 0.  , ..., 0.  , 0.  , 0.  ],
       [1.  , 0.35, 0.  , ..., 0.  , 0.  , 0.  ],
       [0.  , 0.25, 0.  , ..., 0.  , 0.  , 0.  ]])

In [26]:
X_train.shape

(57285, 27)

In [27]:
print(X_train[0])

[1.   0.15 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]


In [28]:
import os
import pandas as pd

data_location = r'D:\Datasets\gender-names\gender_refine-csv.csv'

data = pd.read_csv(data_location)
for i, gender in enumerate(data["Gender"]):
  if gender == 3:
    data.drop(data.index[0], inplace = True)
data['length'] = data.apply(lambda row: len(row.Name), axis=1)

vowels = 'aeiou'
data['no_of_vowels'] = 0
data['percentage_of_vowels'] = 0.0
data['ends_with_vowels'] = 0

for i, names in enumerate(data['Name']):
  count_vowels = 0
  for alpha in names.lower():
    if alpha in vowels:
      count_vowels += 1
  data['no_of_vowels'][i] = count_vowels
  data['percentage_of_vowels'][i] = count_vowels/(len(names))
  if names[len(names)-1] in vowels:
    data['ends_with_vowels'][i] = 1
  else:
    data['ends_with_vowels'][i] = 0
    
maxLength = data['length'].max()

initial = 'abcdefghijklm'
final = 'nopqrstuvwxyz'
dict_alpha = {}
for i in (initial+final):
  if i in initial:
    dict_alpha[i] = float(ord(i) -96 -14)
  if i in final:
    dict_alpha[i] = float(ord(i) -96 -13)
dict_alpha["'"] = 0.0
dict_alpha["-"] = 0.0
dict_alpha[","] = 0.0
dict_alpha[" "] = 0.0
dict_alpha["."] = 0.0
dict_alpha["\\"] = 0.0
dict_alpha["/"] = 0.0
dict_alpha["("] = 0.0
dict_alpha[")"] = 0.0
dict_alpha["["] = 0.0
dict_alpha["]"] = 0.0
dict_alpha["{"] = 0.0
dict_alpha["}"] = 0.0

for i in range(10):
  dict_alpha[str(i)] = 0.0


for i in range(22):
  data["{}st_character".format(i)] = 0

for i, names in enumerate(data['Name']):
  for j in range(maxLength):
    if j < len(names):
      data["{}st_character".format(j)][i] = dict_alpha[names[j].lower()]
    else:
      data["{}st_character".format(j)][i] = 0.0
    
data = data.sample(frac=1).reset_index(drop=True)
y = data['Gender']
data.drop(labels="Gender", axis=1, inplace=True)
data.drop(labels = "Score", axis=1, inplace=True)

data.head(15)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

,Name,length,no_of_vowels,percentage_of_vowels,ends_with_vowels,0st_character,1st_character,2st_character,3st_character,4st_character,...,12st_character,13st_character,14st_character,15st_character,16st_character,17st_character,18st_character,19st_character,20st_character,21st_character
0,Mus'ab,6,0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Sivana,6,0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Savannahmarie,13,0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Akeera,6,0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Waneta,6,0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Ediz,4,0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Mikaylee,8,0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Blakelynn,9,0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Robbert,7,0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Marren,6,0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
data.head()

,Name,length,no_of_vowels,percentage_of_vowels,ends_with_vowels,0st_character,1st_character,2st_character,3st_character,4st_character,...,12st_character,13st_character,14st_character,15st_character,16st_character,17st_character,18st_character,19st_character,20st_character,21st_character
0,Mus'ab,6,0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Sivana,6,0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Savannahmarie,13,0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Akeera,6,0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Waneta,6,0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
data.to_csv(r'C:/Users/Sagar Mishra/temp_csv.csv')